In [1]:
import os
import sys
import re
import datetime
import tempfile
from collections import Counter
import functools
import pickle
import multiprocessing

import seaborn as sns

import numpy as np
import scipy.stats as stats
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster, fclusterdata
from scipy.spatial.distance import cdist, pdist
from scipy import interp
import pandas as pd
pd.set_option('display.max_columns', 250)
# Don't cut off long string
# http://stackoverflow.com/questions/26277757/pandas-to-html-truncates-string-contents
pd.set_option('display.max_colwidth', -1)

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patheffects as PathEffects
from mpl_toolkits.mplot3d import Axes3D
matplotlib.style.use('classic')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (8, 4.5)

from IPython.display import display

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, label_binarize
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
kappa_scorer = metrics.make_scorer(metrics.cohen_kappa_score)
from sklearn.feature_selection import SelectKBest, chi2

from sklearn.cluster import KMeans

In [2]:
import word2vec

In [3]:
%time model = word2vec.load('./PubMed-w2v.bin')

CPU times: user 43.7 s, sys: 12.8 s, total: 56.4 s
Wall time: 58.2 s


In [4]:
model.vocab

array(['</s>', '.', 'of', ..., 'SEDT(AK)', 'Phenidon',
       '[14C]poly(I)-poly(C)'], 
      dtype='<U78')

In [5]:
model.vocab.shape

(2351706,)

In [6]:
model.vectors.shape

(2351706, 200)

In [7]:
model['TP53'].shape

(200,)

In [27]:
model.analogy?

In [42]:
# oncogene + tumour suppressor - tumour suppressor == oncogene
# indexes, metrics = model.analogy(['ABL1', 'TP53'], neg=['BRCA2'])

In [55]:
indexes, metrics = model.analogy(['glioblastoma', 'colon'], neg=['brain'], n=20)

In [56]:
model.generate_response(indexes, metrics).tolist()

[('DLD-1', 0.37927617740198083),
 ('HCT-116', 0.3792706339846247),
 ('SGC-7901', 0.3773898390364688),
 ('HT29', 0.3757760668976403),
 ('SW620', 0.3702126103301677),
 ('SW480', 0.3698747569687583),
 ('NCI-N87', 0.3697115758542201),
 ('TMK-1', 0.3672855981571268),
 ('HCT-15', 0.3643113081923575),
 ('HT-29', 0.36186997431842427),
 ('HCT116', 0.36033603706163),
 ('HCC2998', 0.3595243623015524),
 ('RKO', 0.35786854578089594),
 ('WiDr', 0.3572897604084304),
 ('MGC-803', 0.35647309692111595),
 ('SW-480', 0.35511936292346524),
 ('DLD1', 0.35511841035857244),
 ('LoVo', 0.35487659317909503),
 ('Colo205', 0.35452625027635315),
 ('SW1116', 0.3541507517255692)]

In [29]:
# model['breast'] + model['BRCA1'] - model['prostate']

In [57]:
indexes, metrics = model.cosine('TP53', n=30)
model.vocab[indexes]

array(['k-ras', 'CDKN2A', 'KRAS2', 'K-RAS', 'RB1', 'Tp53', 'KRAS',
       'PIK3CA', 'K-ras', 'CTNNB1', 'FBXW7', 'CDKN2A/p16', 'BRAF', 'HRAS',
       'ARID1A', 'DPC4', 'IDH1/IDH2', 'p16/CDKN2A', 'CDKN2', 'CDH1',
       'NRAS', 'KRAS/BRAF', 'PTEN/MMAC1', 'CHEK2', 'p53-gene', 'TP73',
       'IDH1/2', 'BRAF/NRAS', 'SMAD4', 'AXIN1'], 
      dtype='<U78')